# YOLO Object Detection - Full Inference

This notebook demonstrates full-resolution object detection using YOLO on edge devices.
Requires GPU support and minimum 8GB memory.

## Device Compatibility
- Fully compatible with Jetson AGX Orin
- Compatible with Jetson Nano (with reduced batch size)
- Not recommended for Raspberry Pi

In [ ]:
import sys
sys.path.append('../..')
from base_config import DeviceConfig
from monitoring import DeviceMonitor

# Initialize device configuration
config = DeviceConfig()
monitor = DeviceMonitor()

# Check device compatibility
requirements = {
    "gpu_required": True,
    "min_memory": "8GB",
    "min_cuda_capability": "5.0"
}

if not config.check_notebook_compatibility(requirements):
    raise RuntimeError("Device does not meet minimum requirements for this notebook")

print(f"Device {config.device_type} is compatible with this notebook")
print("Available features:", config.get_device_features())


In [ ]:
# Import required libraries
import torch
from ultralytics import YOLO
import cv2
import numpy as np
from supervision import BoxAnnotator, VideoInfo

# Load optimized model based on device type
model = YOLO('yolov8n.pt')

# Set device-specific parameters
batch_size = config.get_optimized_batch_size()
print(f"Using optimized batch size: {batch_size}")


In [ ]:
# Function for inference
def process_frame(frame):
    results = model(frame)[0]
    return results

# Example usage with monitoring
def run_inference(video_path):
    cap = cv2.VideoCapture(video_path)
    box_annotator = BoxAnnotator()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Get system metrics before inference
        metrics = monitor.collect_metrics()
        
        # Run inference
        results = process_frame(frame)
        
        # Check resource usage
        warnings = monitor.check_thresholds(metrics)
        if warnings:
            print("Resource warnings:", warnings)
            
    cap.release()
